In [4]:
%pip install keras
%pip install tensorflow
%pip install numpy
%pip install pandas

/bin/bash: /home/justyn/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/justyn/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/justyn/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/justyn/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin

In [35]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [36]:
# Load in the dataset
df1 = pd.read_csv("../resources/sentiment_scores.csv")
df2 = pd.read_csv("../resources/raw-dataset.csv")
df = pd.merge(df1, df2, on='row_id')
df.head()

,Unnamed: 0,row_id,text_x,pos,neg,neu,compound,category,rating,label,text_y
0,0,1,love this well made sturdy and very comfor...,0.715,0.000,0.285,0.9558,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,1,2,love it a great upgrade from the original i...,0.491,0.000,0.509,0.8910,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,2,3,this pillow saved my back i love the look and...,0.389,0.000,0.611,0.7906,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,3,4,missing information on how to use it but it i...,0.279,0.079,0.642,0.7227,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,4,5,very nice set good quality we have had the s...,0.281,0.000,0.719,0.7397,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [37]:
# Create a new column called real, if label==CG set to 0, or False, if set to OR set to 1, or True
df['real'] = df['label'].apply(lambda x: 0 if x == 'CG' else 1)
# Randomize the order of the data
df = df.sample(frac=1).reset_index(drop=True)
# Split into train and test data
train = df[:int(len(df)*0.8)]
test = df[int(len(df)*0.8):]

In [38]:
# For training we only need the scores and the label
train_x = train[['pos', 'neg', 'neu', 'compound']]
train_y = train['real']

test_x = test[['pos', 'neg', 'neu', 'compound']]
test_y = test['real']

In [39]:
# Reshape the data to be 3D
train_x = np.array(train_x).reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = np.array(test_x).reshape((test_x.shape[0], 1, test_x.shape[1]))

# change the train data to be a float type
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

In [40]:
# Create the model
model = Sequential()
model.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Train
model.fit(train_x, train_y, epochs=50, batch_size=32, validation_data=(test_x, test_y), verbose=2, shuffle=False)

Epoch 1/50
1011/1011 - 8s - loss: 0.6518 - accuracy: 0.6110 - val_loss: 0.6557 - val_accuracy: 0.6063 - 8s/epoch - 8ms/step
Epoch 2/50
1011/1011 - 8s - loss: 0.6517 - accuracy: 0.6106 - val_loss: 0.6555 - val_accuracy: 0.6064 - 8s/epoch - 8ms/step
Epoch 3/50
1011/1011 - 9s - loss: 0.6515 - accuracy: 0.6108 - val_loss: 0.6553 - val_accuracy: 0.6071 - 9s/epoch - 9ms/step
Epoch 4/50
1011/1011 - 8s - loss: 0.6513 - accuracy: 0.6115 - val_loss: 0.6551 - val_accuracy: 0.6089 - 8s/epoch - 8ms/step
Epoch 5/50
1011/1011 - 8s - loss: 0.6511 - accuracy: 0.6122 - val_loss: 0.6550 - val_accuracy: 0.6109 - 8s/epoch - 8ms/step
Epoch 6/50
1011/1011 - 11s - loss: 0.6509 - accuracy: 0.6123 - val_loss: 0.6548 - val_accuracy: 0.6127 - 11s/epoch - 11ms/step
Epoch 7/50
1011/1011 - 8s - loss: 0.6507 - accuracy: 0.6119 - val_loss: 0.6547 - val_accuracy: 0.6128 - 8s/epoch - 8ms/step
Epoch 8/50
1011/1011 - 8s - loss: 0.6506 - accuracy: 0.6126 - val_loss: 0.6545 - val_accuracy: 0.6136 - 8s/epoch - 8ms/step
Epoch

In [ ]:
# These will be our x data
model.save("trained_model_on_sentiment_data")

In [41]:
# After model training, we want to test our data

predictions = model.predict(test_x)
predictions = [1 if x > 0.5 else 0 for x in predictions]
actual = [x for x in test_y]
comparisons = pd.DataFrame({'prediction': predictions, 'actual': actual})
comparisons['correct'] = comparisons.apply(lambda x: 1 if x['prediction'] == x['actual'] else 0, axis=1)


253/253 [==============================] - 1s 4ms/step


In [42]:
# Load in the dataset
df1 = pd.read_csv("../resources/cleaned_data.csv")
df2 = pd.read_csv("../resources/raw-dataset.csv")
df = pd.merge(df1, df2, on='row_id')
df.head()
df = df.drop(columns=["Unnamed: 0"])

# Create a new column called real, if label==CG set to 0, or False, if set to OR set to 1, or True
df['real'] = df['label'].apply(lambda x: 0 if x == 'CG' else 1)
# Only use columns with float types or int types
df = df.select_dtypes(include=['int64', 'float64'])
# Randomize the order of the data
df = df.sample(frac=1).reset_index(drop=True)
# Split into train and test data
train = df[:int(len(df)*0.8)]
test = df[int(len(df)*0.8):]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40432 entries, 0 to 40431
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   row_id           40432 non-null  int64  
 1   neg              40432 non-null  float64
 2   neu              40432 non-null  float64
 3   pos              40432 non-null  float64
 4   compound         40432 non-null  float64
 5   total_words      40432 non-null  int64  
 6   unique_words     40432 non-null  int64  
 7   noun_count       40432 non-null  float64
 8   adj_count        40432 non-null  float64
 9   verb_count       40432 non-null  float64
 10  adv_count        40432 non-null  float64
 11  pro_count        40432 non-null  float64
 12  pre_count        40432 non-null  float64
 13  con_count        40432 non-null  float64
 14  art_count        40432 non-null  float64
 15  nega_count       40432 non-null  float64
 16  aux_count        40432 non-null  float64
 17  noun_percent

In [46]:
# For training we only need the scores and the label
train_x = train.drop(columns=["real"])
train_y = train['real']

test_x = test.drop(columns=["real"])
test_y = test['real']

In [47]:
# Reshape the data to be 3D
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

train_x = np.array(train_x).reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = np.array(test_x).reshape((test_x.shape[0], 1, test_x.shape[1]))

# change the train data to be a float type
# Create the model
model = Sequential()
model.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train
history = model.fit(train_x, train_y, epochs=50, batch_size=32, validation_data=(test_x, test_y), shuffle=False)

Epoch 1/50
1011/1011 [==============================] - 11s 9ms/step - loss: 0.6932 - accuracy: 0.5059 - val_loss: 0.6915 - val_accuracy: 0.4973
Epoch 2/50
1011/1011 [==============================] - 9s 9ms/step - loss: 0.6921 - accuracy: 0.5079 - val_loss: 0.6917 - val_accuracy: 0.4946
Epoch 3/50
1011/1011 [==============================] - 10s 10ms/step - loss: 0.6927 - accuracy: 0.5021 - val_loss: 0.6922 - val_accuracy: 0.5051
Epoch 4/50
1011/1011 [==============================] - 10s 10ms/step - loss: 0.6931 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.4977
Epoch 5/50
1011/1011 [==============================] - 10s 10ms/step - loss: 0.6926 - accuracy: 0.5019 - val_loss: 0.6919 - val_accuracy: 0.4991
Epoch 6/50
1011/1011 [==============================] - 10s 10ms/step - loss: 0.6924 - accuracy: 0.5030 - val_loss: 0.6918 - val_accuracy: 0.5054
Epoch 7/50
1011/1011 [==============================] - 11s 10ms/step - loss: 0.6928 - accuracy: 0.5024 - val_loss: 0.6920 - va

In [ ]:
# After model training, we want to test our data

predictions = model.predict(test_x)
predictions = [1 if x > 0.5 else 0 for x in predictions]


253/253 [==============================] - 1s 2ms/step


In [ ]:
actual = [x for x in test_y]
comparisons = pd.DataFrame({'prediction': predictions, 'actual': actual})
comparisons['correct'] = comparisons.apply(lambda x: 1 if x['prediction'] == x['actual'] else 0, axis=1)
comparisons

,prediction,actual,correct
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,1,1,1
...,...,...,...
8082,0,0,1
8083,0,0,1
8084,0,0,1
8085,0,1,0


In [ ]:
def classify_false_positives(x):
  # True Positive, we correctly identified a human
  if x['actual'] == 1 and x['prediction'] == 1:
    return 'TP'
  # True Negative, we correctly identified a computer
  elif x['actual'] == 0 and x['prediction'] == 0:
    return 'TN'
  # False Negative, it was a human but we said it was a bot
  elif x['actual'] == 1 and x['prediction'] == 0:
    return 'FN'
  # False Positive, it was a bot, but we said it was a human
  else:
    return 'FP'

# Create a new column called classification
comparisons['classification'] = comparisons.apply(classify_false_positives, axis=1)
print(comparisons['classification'].value_counts())

TN    3594
FN    3464
TP     580
FP     449
Name: classification, dtype: int64
